## Import Libraries

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Data Preprocessing

In [3]:
df = pd.read_csv("laptop_price.csv", encoding="latin-1")

In [4]:
df

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.8kg,638.00
1299,1317,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16GB,512GB SSD,Intel HD Graphics 520,Windows 10,1.3kg,1499.00
1300,1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00
1301,1319,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,764.00


In [5]:
df = df.drop(["Product", "laptop_ID"], axis=1)

In [6]:
df = df.join(pd.get_dummies(df["Company"]))
df = df.drop(["Company"], axis=1)

In [7]:
df = df.join(pd.get_dummies(df["TypeName"]))
df = df.drop(["TypeName"], axis=1)

In [8]:
df.ScreenResolution = df.ScreenResolution.str.split(" ").apply(lambda x: x[-1])

df["Screen Width"] = df.ScreenResolution.str.split("x").apply(lambda x: x[0])
df["Screen Height"] = df.ScreenResolution.str.split("x").apply(lambda x: x[1])

df["Screen Width"] = df["Screen Width"].astype("int")
df["Screen Height"] = df["Screen Height"].astype("int")

df = df.drop(["ScreenResolution"], axis=1)

In [9]:
df["CPU Brand"] = df.Cpu.str.split(" ").apply(lambda x: x[0])
df["CPU Frequency"] = df.Cpu.str.split(" ").apply(lambda x: x[-1])

In [10]:
df["CPU Frequency"] = df["CPU Frequency"].apply(lambda x: x[:-3])
df["CPU Frequency"] = df["CPU Frequency"].astype("float")

In [11]:
cpu_brands = pd.get_dummies(df["CPU Brand"])

cpu_brands.columns = [column + " CPU" for column in cpu_brands.columns] 

df = df.join(cpu_brands)

df = df.drop(["CPU Brand"], axis=1)
df = df.drop(["Cpu"], axis=1)

In [12]:
df.Ram = df.Ram.apply(lambda x: x[:-2])
df.Ram = df.Ram.astype("int")

In [22]:
df["Memory Type"] = df.Memory.str.split(" ").apply(lambda x: x[1])

df = df.join(pd.get_dummies(df["Memory Type"]))
df = df.drop(["Memory Type"], axis=1)

In [34]:
df["Memory MB"] = df.Memory.str.split(" ").apply(lambda x: x[0])
df["Memory MB"] = df["Memory MB"].apply(lambda x: float(x[:-2]) * 1000 if x[-2:] == "GB" else float(x[:-2]) * 1000000)
df["Memory MB"] = df["Memory MB"].astype("float")

In [36]:
df = df.drop(["Memory"], axis=1)

In [37]:
df

,Inches,Ram,Gpu,OpSys,Weight,Price_euros,Acer,Apple,Asus,Chuwi,...,Screen Height,CPU Frequency,AMD CPU,Intel CPU,Samsung CPU,Memory MB,Flash,HDD,Hybrid,SSD
0,13.3,8,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69,False,True,False,False,...,1600,2.3,False,True,False,128000.0,False,False,False,True
1,13.3,8,Intel HD Graphics 6000,macOS,1.34kg,898.94,False,True,False,False,...,900,1.8,False,True,False,128000.0,True,False,False,False
2,15.6,8,Intel HD Graphics 620,No OS,1.86kg,575.00,False,False,False,False,...,1080,2.5,False,True,False,256000.0,False,False,False,True
3,15.4,16,AMD Radeon Pro 455,macOS,1.83kg,2537.45,False,True,False,False,...,1800,2.7,False,True,False,512000.0,False,False,False,True
4,13.3,8,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60,False,True,False,False,...,1600,3.1,False,True,False,256000.0,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,14.0,4,Intel HD Graphics 520,Windows 10,1.8kg,638.00,False,False,False,False,...,1080,2.5,False,True,False,128000.0,False,False,False,True
1299,13.3,16,Intel HD Graphics 520,Windows 10,1.3kg,1499.00,False,False,False,False,...,1800,2.5,False,True,False,512000.0,False,False,False,True
1300,14.0,2,Intel HD Graphics,Windows 10,1.5kg,229.00,False,False,False,False,...,768,1.6,False,True,False,64000.0,True,False,False,False
1301,15.6,6,AMD Radeon R5 M330,Windows 10,2.19kg,764.00,False,False,False,False,...,768,2.5,False,True,False,1000000.0,False,True,False,False
